### Social Media Agentic Team

Experiment with an agentic team of social media agents. Given a topic / social media post idea, multiple agents will kick off to generate content for specific social media platforms.

In [52]:
import os 
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv()

from IPython.display import display, Markdown
import base64

from utils.llm import chat_completion

from openai import OpenAI
client = OpenAI()

from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

from pydantic import BaseModel, Field
from typing import Annotated, Sequence, TypedDict, List, Dict

### Create tools for each agent

#### 1. Image generation

In [14]:
# name image - when saving, we will ask AI to name the image

def name_image(image_prompt: str):

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You assist with naming images. Given an image prompt, you will return a name for the image to save it to a png. Do not add the extension to the name."},
            {"role": "user", "content": image_prompt}
        ]
    )

    generated_name = completion.choices[0].message.content
    print(f'Generated name: {generated_name}')

    return generated_name

# generate image

def generate_image(image_prompt: str):

    response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        size="1024x1024",
        quality="hd",
        style="vivid",
        n=1,
        response_format = "b64_json" #url
    )

    revised_prompt = response.data[0].revised_prompt
    #url = response.data[0].url
    base64_image = response.data[0].b64_json

    print(f'Successfully generated image')

    return revised_prompt, base64_image

# save image

def save_image(image_name: str, base64_image: str):

    img_data = base64.b64decode(base64_image)
    path = f"social_media_agentic_team_temp/{image_name}.png"

    with open(path, "wb") as f:
        f.write(img_data)

    print(f'Successfully saved image to {path}')

    return path

# image tool

@tool
def image_generation_tool(image_prompt: str):

    """
    Generate an image given a description.
    """

    revised_prompt, base64_image = generate_image(image_prompt)
    image_name = name_image(revised_prompt)
    path = save_image(image_name, base64_image)

    return f"Image generated and saved to {path}"

Test `DALL-E-3` image generation.

In [15]:
image_prompt = "Never before seen demand for GPUs"
image_generation_tool(image_prompt)

Successfully generated image
Generated name: GPU_Frenzy_in_Hardware_Store
Successfully saved image to social_media_agentic_team_temp/GPU_Frenzy_in_Hardware_Store.png


'Image generated and saved to social_media_agentic_team_temp/GPU_Frenzy_in_Hardware_Store.png'

In [22]:
path = 'social_media_agentic_team_temp/GPU_Frenzy_in_Hardware_Store.png'

markdown_text = f"""

## Test Markdown with Image

Here is the image:

<img src="{path}" width="450"/>

End of test markdown.
"""

display(Markdown(markdown_text))




## Test Markdown with Image

Here is the image:

<img src="social_media_agentic_team_temp/GPU_Frenzy_in_Hardware_Store.png" width="450"/>

End of test markdown.


#### 2. Perplexity AI Research

In [48]:
model_name_pplx = "llama-3.1-sonar-large-128k-online"
pplx_client = OpenAI(api_key=os.getenv('PPLX_API_KEY'), base_url="https://api.perplexity.ai")
temperature = 0.0
system_msg_pplx = "You are an expert social media content creator. You provide helpful, professional advice to the user."

def perplexity_research(user_message: str):

    response_pplx = chat_completion(
            messages=[
                {"role": "system", "content": system_msg_pplx},
            {"role": "user", "content": user_message}
        ],
        model_name=model_name_pplx,
        temperature=temperature,
        client=pplx_client
    )

    response = response_pplx.choices[0].message.content
    citations = response_pplx.citations

    return response, citations

@tool
def perplexity_research_tool(user_message: str):
    """
    Use Perplexity AI to research a topic.
    """

    response, citations = perplexity_research(user_message)
    pplx_output = f"Insights from Perplexity AI:\n\n{response}\n\nCitations:\n{citations}"

    return pplx_output

Test perplexity research tool

In [49]:
pplx_research_question = "Who is most likely to compete with Nvidia in the GPU market?"
pplx_test_output = perplexity_research_tool(pplx_research_question)

display(Markdown(pplx_test_output))

Insights from Perplexity AI:

In the GPU market, the primary competitors to Nvidia are:

## Advanced Micro Devices (AMD)
AMD is Nvidia's most significant and longstanding competitor in the GPU market. AMD offers cost-effective graphics cards, such as its Radeon series, which compete directly with Nvidia's products in terms of performance and price. Despite Nvidia's dominant market share, AMD has managed to maintain a considerable portion of the market, particularly in the mid-range segment[1][2][3].

## Intel
Intel, although newer to the GPU market, is also a notable competitor. Intel has launched its Xe Graphics and ARC GPUs, leveraging its expertise in integrated circuits to develop competitive GPU offerings. However, Intel's current market share in the GPU sector is minimal, and it faces significant challenges in gaining traction against Nvidia and AMD[1][2][3].

## Other Notable Players
While not as prominent in the GPU market, other companies like Sapphire Technology, which specializes in customized AMD-based graphics cards, and startups focusing on specific AI chip solutions, also contribute to the competitive landscape. However, their impact is currently more niche compared to the broader market influence of AMD and Intel[2][4].

In summary, AMD is the most likely and significant competitor to Nvidia in the GPU market, given its established presence and competitive offerings. Intel, though still developing its market presence, also poses a potential challenge, especially with its integrated circuit expertise.

Citations:
['https://www.nerdwallet.com/article/investing/nvidia-competitors', 'https://www.cheddarflow.com/blog/exploring-the-top-nvidia-competitors-in-2023/', 'https://www.techradar.com/computing/gpu/nvidia-now-owns-88-of-the-gpu-market-but-that-might-not-be-a-bad-thing-yet', 'https://fortune.com/2024/07/02/nvidia-competition-ai-chip-gpu-startups-analysts/', 'https://www.techpowerup.com/forums/threads/nvidias-gpu-market-share-hits-90-in-q4-2024-gets-closer-to-full-monopoly.329842/']

### 2. Prompts

In [ ]:
planning_system_msg = "You are an expert social media content creator. You provide helpful, professional advice to the user. You are given a topic and a social media platform. You will provide a plan for the content to be created."

planning_instruction = """
The user wants to create a post about the following topic: 
<topic>{topic}</topic>.

The selected social media platform is: 
<platform>{platform}</platform>.

Background information about the user and the audience: 
<background_info>{background_info}</background_info>.

Return a plan for the content to be created. You can use the following tools:
<tools>{tools}</tools>.
"""

creation_system_msg = "You are an expert social media content creator. You are given tools and a plan for the content to be created. Using your expertise and reasoning, you will create the content."

creation_instruction = """
The user wants to create a post about the following topic: 
<topic>{topic}</topic>.

The selected social media platform is: 
<platform>{platform}</platform>.

Background information about the user and the audience: 
<background_info>{background_info}</background_info>.

The plan for the content to be created is: 
<plan>{plan}</plan>.

You have been provided with the following tools:
<tools>{tools}</tools>.

Once you've done your research, called the necessary tools generate and return with the content. The social media post should be written in markdown format. For the visual content, embed the image path in the markdown.

Remember to adhere to the instructions given.
"""

### 3. Structured Outputs

In [53]:
class SocialMediaPostStep(BaseModel):
    step_number: int = Field(description="The number of the step")
    step_description: str = Field(description="A description of the step")
    step_reasoning: str = Field(description="The reasoning behind the step")
    step_tool: str = Field(description="The tool to be used for the step")

class SocialMediaPostPlan(BaseModel):
    steps: List[SocialMediaPostStep] = Field(description="The steps to create the social media post")